In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-14B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)


/home/jovyan/hrishik/playground/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 8/8 [00:06<00:00,  1.21it/s]


In [2]:
from transformers import AutoTokenizer, AutoModel

tokenizer_emb = AutoTokenizer.from_pretrained("Qwen/Qwen3-Embedding-8B")
model_emb = AutoModel.from_pretrained("Qwen/Qwen3-Embedding-8B").to('cuda')

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.21s/it]


In [ ]:
# from transformers import StoppingCriteria, StoppingCriteriaList

# class StopAtPunctuation(StoppingCriteria):
#     def __call__(self, input_ids, scores, **kwargs):
#         text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
#         return '</ai>' in text

# stop_criteria = StoppingCriteriaList([StopAtPunctuation()])

5120

In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
import torch
# Enable PyTorch 2.x compilation
model = torch.compile(model, mode="reduce-overhead")  # or mode="reduce-overhead" for faster iteration

In [14]:
import torch
import torch.nn.functional as F

def unregister_all_hooks(handles):
    for h in handles:
        h.remove()
    handles.clear()

handles = []

device = 'cuda'
hidden_size = model.config.hidden_size
scale = 2.0

# Initialize normalized vector to zero first
v_normalized = torch.zeros(hidden_size, device=device, dtype=torch.bfloat16)

def random_hook(module, input, output):
    hidden = output[0]
    v_scaled = (v_normalized * scale).to(hidden.device)
    return (hidden + v_scaled,) + output[1:]

for layer in model.model.layers:
    handles.append(layer.register_forward_hook(random_hook))

# prompt = """
# <|im_start|>system
# What follows is a conversation between a person and their assistant.
# <|im_end|>
# <|im_start|>user
# Do you think we exist because of a higher power?
# <|im_end|>
# <|im_start|>assistant
# """

prompt = """
"Bartenders cleaning up after a busy night, having observed human nature at its most uninhibited for hours.","<|im_start|>bartender1
What's the most pathetic pickup line you overheard tonight?
<|im_end|>"
<|im_start|>bartender2
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Original output with hooks inactive (v_normalized = 0)
orig_outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    eos_token_id=tokenizer.convert_tokens_to_ids("<|im_end|>")
)
generated_tokens = orig_outputs[0][inputs['input_ids'].shape[1]:]
text_orig = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print("Original output:", text_orig)

# Set v to random and normalize
v = torch.randn(hidden_size, device=device, dtype=torch.bfloat16).unsqueeze(0).unsqueeze(0)
v_norm = v.norm(p=2)
v_normalized = (v / v_norm).to(torch.bfloat16).to(device)

# Corrupted output with hooks active
hook_outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    eos_token_id=tokenizer.convert_tokens_to_ids("<|im_end|>")
)
generated_tokens_hook = hook_outputs[0][inputs['input_ids'].shape[1]:]
text_with_hooks = tokenizer.decode(generated_tokens_hook, skip_special_tokens=True)
print("Corrupted output:", text_with_hooks)

# Compute embeddings
emb_inputs_orig = tokenizer_emb(text_orig, return_tensors="pt").to(device)
emb_inputs_hook = tokenizer_emb(text_with_hooks, return_tensors="pt").to(device)


my_inputs_1 = tokenizer_emb("Nope, I'm barely employable today.", return_tensors="pt").to(device)
# my_inputs_2 = tokenizer_emb("adgadsfasdg", return_tensors="pt").to(device)

with torch.no_grad():
    emb_orig = model_emb(**emb_inputs_orig).last_hidden_state.mean(dim=1)
    emb_hook = model_emb(**emb_inputs_hook).last_hidden_state.mean(dim=1)
    # my_emb_1 = model_emb(**my_inputs_1).last_hidden_state.mean(dim=1)
    # my_emb_2 = model_emb(**my_inputs_2).last_hidden_state.mean(dim=1)


cos_sim = F.cosine_similarity(emb_orig, emb_hook)
# cos_sim = F.cosine_similarity(emb_orig, my_emb_1)
# cos_sim = F.cosine_similarity(my_emb_1, my_emb_2)
print("Cosine similarity of embeddings:", cos_sim.item())

unregister_all_hooks(handles)

Original output: "Oh, there was this one guy who said, 'Are you a magician? Because whenever I look at you, everyone else disappears.' I mean, really? That's about as subtle and original as a signpost in a desert!"
Corrupted output: As a bartender, I've heard my fair share of pickup lines. One of the more humorous and somewhat pathetic ones goes something like this: "Excuse me, do you have a map? Because I just got lost in your eyes." While it
Cosine similarity of embeddings: 0.8714981079101562
